#### Package import

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

from urllib.request import urlopen


print('Libraries imported.')

Libraries imported.


#### Previous code from part 1

In [32]:

# import of wikipedia data, must be converted to a dataframe format
wikiLink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = urlopen(wikiLink).read()
soup = BeautifulSoup(page, 'html.parser')

table = soup.find_all('table')[0] 

df = pd.read_html(str(table))[0]

# drop the lines for which the borough is not assigned
index2del = df[df['Borough']=='Not assigned'].index

df.drop(index = index2del, inplace = True)
df.reset_index(inplace = True, drop = True)

# assign borough name for undefined neighbourhood
index2mod = df[df['Neighbourhood']=='Not assigned'].index
df.iloc[index2mod, 2] = df.iloc[index2mod, 1]


# creation of a list that references duplicates
duplicates = df[df['Postcode'].duplicated()]
index2del = duplicates.index # lines indexes must be deleted from the df list
df.drop(index = index2del, inplace = True)

# reset of index
df.reset_index(inplace = True, drop = True)
duplicates.reset_index(inplace = True, drop = True)

# loop over the rows of df (which contain unique postcode values)
for index1, row in df.iterrows():
    
    # finds the indexes of the duplicates
    indexFind = duplicates[row['Postcode'] == duplicates['Postcode']].index
    
    # loop over the indexes
    for i in indexFind:
        # appends the neighbourhood string field
        df.iloc[index1, 2] = df.iloc[index1, 2] + ',' + duplicates.iloc[i,2]

## Part 2

#### Import of csv data

In [44]:

link = "http://cocl.us/Geospatial_data"
geoData = pd.read_csv(link)

geoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Assigning the corresponding geospatial data

In [45]:
#Initializing of the new series
df['Latitude'] = 0.0
df['Longitude'] = 0.0

# loop over df rows
for index, row in df.iterrows():
    postCode = row['Postcode']
    indexFind = geoData[geoData["Postal Code"] == postCode].index

    df.iloc[index, 3] = geoData.iloc[indexFind,1].values
    df.iloc[index, 4] = geoData.iloc[indexFind,2].values
    
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
